<a href="https://colab.research.google.com/github/donbcolab/AIE3/blob/main/paligemma_cnmc_finetune_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [Gemma 2B E2E Fine Tuning...](https://colab.research.google.com/gist/donbr/505c3101276cfdc5360b2367af9da054/fine_tuning_gemma-2b_instruct_for_summarization_assignment_version_jun14.ipynb) -  inspirational quotes from a tiny LM!!!
using all the good stuff from Hugging Face:
1. Load and Quantize the base Model (bitsandbytes) 🔍📦
2. Load and Prepare Data (datasets) 📂📊
3. Fine Tuning Configuration (peft) ⚙️🛠️
4. Train the adapter Model (trl) 🏋️‍♂️🤖
5. Model Evaluation and Deployment (peft - merge and unload) 📈🚀)

### Setting Up

In [ ]:
base_model_name = "google/paligemma-3b-pt-224"
adapter_version = "paligemma-cnmc-ft"
adapter_model_name = f"dwb2023/{adapter_version}"

In [ ]:
!pip install -qU bitsandbytes datasets accelerate loralib peft transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import(
    PaliGemmaForConditionalGeneration, PaliGemmaProcessor, TrainingArguments,
    Trainer, BitsAndBytesConfig, BitsAndBytesConfig
)
from datasets import load_dataset, DatasetDict

In [ ]:
# verify gpu is available
torch.cuda.is_available()

True

In [ ]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

In [ ]:
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

## Load Dataset

In [ ]:
# Load CNMC Dataset
ds = load_dataset("dwb2023/cnmc-leukemia-2019", split="train")

Generating train split:   0%|          | 0/10661 [00:00<?, ? examples/s]

In [ ]:
# Define the percentage you want to retrieve (e.g., 10%)
percentage = 0.10

# Use train_test_split to get the subset
cnmc_ds = ds.train_test_split(test_size=percentage)["test"]

cols_remove = ["subject_id", "image_number", "cell_count", "class_label",
               "fold", "original_image_name", "relative_file_path"]
cnmc_ds = cnmc_ds.remove_columns(cols_remove)

In [ ]:
# create train test split with test_size=0.2
train_ds = cnmc_ds.train_test_split(test_size=0.2)

# create test val split
test_val_ds = train_ds["test"].train_test_split(test_size=0.5)

cnmc_ds_dict = DatasetDict({
    "train" : train_ds["train"],
    "test" : test_val_ds["test"],
    "validation" : test_val_ds["train"]
})

cnmc_ds_dict

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 853
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 107
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 107
    })
})

## Collate Data

In [ ]:
processor = PaliGemmaProcessor.from_pretrained(base_model_name)

preprocessor_config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

In [ ]:
from PIL import Image
import io

def collate_fn(batch):
    if not isinstance(batch, list):
        batch = [batch]

    texts = ["Are these cells healthy or cancerous?" for _ in batch]
    labels = [example['label'] for example in batch]

    images = []
    for example in batch:
        if 'image' in example:
            if isinstance(example['image'], dict) and 'bytes' in example['image']:
                img_bytes = example['image']['bytes']
            elif isinstance(example['image'], (bytes, bytearray)):
                img_bytes = example['image']
            else:
                raise ValueError(f"Unexpected image format: {type(example['image'])}")

            img = Image.open(io.BytesIO(img_bytes)).convert("RGB")
            images.append(img)
        else:
            raise KeyError("'image' key not found in example")

    tokens = processor(text=texts, images=images, suffix=labels,
                       return_tensors="pt", padding="longest")

    tokens = tokens.to(torch.bfloat16).to(device)
    return tokens

## Load and Quatize the base Model (bitsandbytes)

In [ ]:
from transformers import BitsAndBytesConfig
from peft import get_peft_model, LoraConfig

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_type=torch.bfloat16
)

lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)
model = PaliGemmaForConditionalGeneration.from_pretrained(base_model_name, quantization_config=bnb_config, device_map={"":0})
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
#trainable params: 11,298,816 || all params: 2,934,634,224 || trainable%: 0.38501616002417344


Unused kwargs: ['bnb_4bit_compute_type']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/62.6k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

trainable params: 11,298,816 || all params: 2,934,765,296 || trainable%: 0.3850


## Train the Adapter Model (trl)

In [ ]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PaliGemmaForConditionalGeneration(
      (vision_tower): SiglipVisionModel(
        (vision_model): SiglipVisionTransformer(
          (embeddings): SiglipVisionEmbeddings(
            (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
            (position_embedding): Embedding(256, 1152)
          )
          (encoder): SiglipEncoder(
            (layers): ModuleList(
              (0-26): 27 x SiglipEncoderLayer(
                (self_attn): SiglipAttention(
                  (k_proj): lora.Linear4bit(
                    (base_layer): Linear4bit(in_features=1152, out_features=1152, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Identity()
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=1152, out_features=8, bias=False)
                    )
                    (lora_B): Mo

In [ ]:
model.config

PaliGemmaConfig {
  "_name_or_path": "google/paligemma-3b-pt-224",
  "architectures": [
    "PaliGemmaForConditionalGeneration"
  ],
  "bos_token_id": 2,
  "eos_token_id": 1,
  "hidden_size": 2048,
  "ignore_index": -100,
  "image_token_index": 257152,
  "model_type": "paligemma",
  "pad_token_id": 0,
  "projection_dim": 2048,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "float32",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method": "bitsandbytes"
  },
  "text_config": {
    "hidden_size": 2048,
    "intermediate_size": 16384,
    "model_type": "gemma",
    "num_attention_heads": 8,
    "num_hidden_layers": 18,
    "num_image_tokens": 256,

In [ ]:
from trl import SFTConfig

max_seq_length = 128
output_dir = adapter_version

args=SFTConfig(
    output_dir = output_dir,
    max_steps=50,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    logging_steps=5,
    eval_strategy="steps",
    eval_steps=5,
    learning_rate=2e-5,
    max_seq_length=max_seq_length,
    bf16=True,
    optim="adamw_hf",
    report_to=["tensorboard"],
)

In [ ]:
cnmc_ds_dict

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 853
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 107
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 107
    })
})

In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
  model=model,
  train_dataset=cnmc_ds_dict['train'],
  eval_dataset=cnmc_ds_dict['validation'],
  data_collator=collate_fn,
  args=args
)

Map:   0%|          | 0/853 [00:00<?, ? examples/s]

KeyError: None

In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub(f"dwb2023/{output_dir}")

### Temporary code below to assist with debugging Image Key error

In [ ]:
cnmc_ds_dict['train'][0].keys()